Verinin anlatılması, açıklanması.

Kullandığım veri seti Online Payments Fraud Detection Dataseti. Veri seti yapılan para işlemlerinden hangisinin dolandırıcılık işlemi olup olmadığını gösteren bir veri seti.  

Veri setindeki özellikler:

    1- step: işlem süresini saat cinsinden ifade eder.
    2- type: online işlemin tipi.
    3- amount: işlemin miktarı.
    4- nameOrig: işlemi başlatan müşterinin adı.
    5- oldbalanceOrg: işlem öncesi hesap bakiyesi.
    6- newbalanceOrig: işlem sonrası hesap bakiyesi.
    7- nameDest: işlemi alan hesabın adı.
    8- oldbalanceDest: işlemin ulaştığı hesabın işlem öncesi bakiyesi.
    9- newbalanceDest: işlemin ulaştığı hesabın işlem sonrası bakiyesi.
    10- isFraud: dolandırıcılık işlemi olup olmadığı.

Bu veri setindeki asıl amaç 1 den 9 a kadar verilen özellikler ile yapılan işlemin dolandırıcılık işlemi olup olmadığının tahminlenmesi.

In [6]:
#EDA - Keşifsel Veri Analizi
import pandas as pd

#datanın pandas kütüphanesi yardımıyla okunması
data = pd.read_csv("/home/tugberk/Downloads/archive/PS_20174392719_1491204439457_log.csv")

In [7]:
#Verinin ilk 5 satırının incelenmesi
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


Veriden elde edilen sonuçların eklenmesi, değerlendirilmesi.

In [ ]:
#Data Preprocessing- Veri Ön İşleme

Data üzerinde neler yapıldı, neden yapıldı açıklanması.

In [ ]:
#Algoritma seçimi ve Hiperparametre Optimizasyonu

Hangi algoritma neden seçildi, seçilen 2 algoritmadan hangisi nasıl sonuç çıkardı neden iyi/kötü sonuç çıkardı.

In [ ]:
#Model Evaluation

Model üzerinde değerlendirme algoritmaları kullanarak sonuçların karşılaştırılması.